<a href="https://colab.research.google.com/github/veroorli/ProjetProg/blob/master/TP2_SUJET_SEUL_cube_rollup_over.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

version du 30/09/2021

# BDLE TP2 : Requêtes multidimensionnelles
### Group By Cube et Group by Rollup 
### Fenêtres: Over et Partition by

## Introduction

On ajuste l'environnement d'exécution des requêtes à la taille du cluster (8 coeurs) qu'on utilise.

In [ ]:
# on utilise 8 partitions au lieu de 200 par défaut
spark.conf.set("spark.sql.shuffle.partitions", "8")

## Les données

On propose 2 possibilités pour disposer des données du TP:
  * Lire un fichier préalablement stocké dans le système de fichier
  * Générer un exemple de données

### La table Visite

In [ ]:
import pyspark.sql.functions as fct

visite_attrs = ["photoID",	"personID",	"date",	"lon",	"lat", "note"]
visite_tuples = [("p1",	"Bob",	"03/09/2020",	41.5,	12.8, 3),
                 ("p2",	"Alice", "01/09/2020",	41.6,	12.8, 5),
                 ("p3",	"Bob",	"04/09/2020",	41.6,	13.2, 1),
                 ("p4",	"Bob",	"04/09/2020",	40.1,	12.0, 2),
                 ("p5",	"Alice", "04/09/2020",	40.1,	12.0, 2),
                 ("p6",	"Alice", "05/09/2020",	41.6,	12.7, 4),
                 ("p7",	"Alice", "05/10/2020",	41.8, 12.8, 4),
                 ("p8",	"Carole", "25/12/2019",	30.1, 10.1, 2),
                 ("p9",	"David", "25/12/2019",	30.1, 10.1, 1),
                 ("p10","Eva", "25/12/2019",	30.1, 10.1, 5),
                 ("p11","Eva", "26/12/2019",	31.1, 10.1, 3),
                 ("p12","Alice", "01/02/2020",	32.1, 12.1, 3),
                 ("p13","Bob", "01/02/2020",	32.1, 12.1, 5),
                 ("p14","Carole", "01/02/2020",	32.1, 12.1, 2),
                 ("p15",	"Carole", "11/11/2019",	49.1, 10.1, 1),
                 ("p16",	"Alice", "25/12/2019",	30.1, 10.1, 4),
                 ("p17",	"Alice", "02/02/2020",	49.1, 10.1, 5),
                ]
visite = spark.createDataFrame(visite_tuples, visite_attrs)

# convertir l'attribut date pour qu'il soit de type Date 
# visite = visite.selectExpr("photoID", "personID", "lon", "lat", "to_date(date, 'dd/MM/yyyy') as date")
visite = visite.withColumn("date", fct.to_date("date", "dd/MM/yyyy"))

visite.persist().count()
visite.createOrReplaceTempView("Visite")

# Deux affichages possibles : display ou show

# display affiche toutes les lignes dans un tableau de 7 lignes de hauteur. Pour voir toutes les lignes, agrandir la zone du tableau ou sélectionner une cellule et scroller
display(visite)

# invoquer show(n) pour afficher n lignes :
# visite.show(100)

photoID,personID,date,lon,lat,note
p1,Bob,2020-09-03,41.5,12.8,3
p2,Alice,2020-09-01,41.6,12.8,5
p3,Bob,2020-09-04,41.6,13.2,1
p4,Bob,2020-09-04,40.1,12.0,2
p5,Alice,2020-09-04,40.1,12.0,2
p6,Alice,2020-09-05,41.6,12.7,4
p7,Alice,2020-10-05,41.8,12.8,4
p8,Carole,2019-12-25,30.1,10.1,2
p9,David,2019-12-25,30.1,10.1,1
p10,Eva,2019-12-25,30.1,10.1,5


Une requête directement exprimée en SQL :

In [ ]:
%sql
select *
from Visite 
where personID='Bob'

photoID,personID,date,lon,lat,note
p1,Bob,2020-09-03,41.5,12.8,3
p3,Bob,2020-09-04,41.6,13.2,1
p4,Bob,2020-09-04,40.1,12.0,2
p13,Bob,2020-02-01,32.1,12.1,5


### La vue DateDimension

On définit une requête pour extraire le jour, le mois et l'année de chaque date distincte. On nomme cette requête.
Une requête nommée est une vue temporaire.

In [ ]:
%sql
create or replace temporary view DateDimension as
select distinct date, day(date) as jour, month(date) as mois, year(date) as annee
from Visite;

select *
from DateDimension
order by annee, mois, jour;

date,jour,mois,annee
2019-11-11,11,11,2019
2019-12-25,25,12,2019
2019-12-26,26,12,2019
2020-02-01,1,2,2020
2020-02-02,2,2,2020
2020-09-01,1,9,2020
2020-09-03,3,9,2020
2020-09-04,4,9,2020
2020-09-05,5,9,2020
2020-10-05,5,10,2020


### La table Place

In [ ]:
place_attr = ["lon","lat", "ville", "pays"]

place_tuples = [(41.5, 12.8, "Aix", "France"),
                (41.6, 12.8, "Aix", "France"),
                (41.6, 12.7, "Nice", "France"),
                (41.8, 12.8, "Marseille", "France"),
                (41.6, 13.2, "Rome", "Italie"),
                (40.1, 12.0, "Oslo", "Norvege"),
                (30.1, 10.1, "Paris", "France"),                
                (31.1, 10.1, "StDenis", "France"),                
                (32.1, 12.1, "Lille", "France"),   
                (49.1, 10.1, "Pau", "France") ]

place = spark.createDataFrame(place_tuples, place_attr )
place.persist().count()
place.createOrReplaceTempView("Place")
display(place)

lon,lat,ville,pays
41.5,12.8,Aix,France
41.6,12.8,Aix,France
41.6,12.7,Nice,France
41.8,12.8,Marseille,France
41.6,13.2,Rome,Italie
40.1,12.0,Oslo,Norvege
30.1,10.1,Paris,France
31.1,10.1,StDenis,France
32.1,12.1,Lille,France
49.1,10.1,Pau,France


### La table Profession

In [ ]:
profession_attr = ["personID","profession"]

profession_tuples = [("Alice", "Architecte"),
                     ("Bob", "Data science"),
                     ("Carole", "Data science"),
                     ("David", "Architecte"),
                     ("Eva", "Vendeuse"),
                     ("Franck", "Vendeur"),
                     ("Greta", "Economiste")
                    ]

profession = spark.createDataFrame(profession_tuples, profession_attr )
profession.persist().count()
profession.createOrReplaceTempView("Profession")
display(profession)

personID,profession
Alice,Architecte
Bob,Data science
Carole,Data science
David,Architecte
Eva,Vendeuse
Franck,Vendeur
Greta,Economiste


## Les requêtes d'analyse

### Les Faits détaillés avec leurs dimensions

La table Visite est la table de faits. Les dimensions des visites sont: 
* une dimension géographique : position gps (lon,lat), ville, pays
* une dimension temporelle sur la date :  jour, mois, année
* une dimension sur la profession : personID, profession

Une jointure associe les Visites avec leurs dimensions.

In [ ]:
%sql 
create or replace temp view VisitesDetail as
Select v.photoID, p.personID, p.profession, v.date, d.jour, d.mois, d.annee, v.lon, v.lat, ville, pays,  v.note
From Visite v, Place p, DateDimension d, Profession p
Where v.lon = p.lon and v.lat = p.lat 
And v.date = d.date
And v.personId = p.personId;


select * 
from VisitesDetail
order by pays, ville

photoID,personID,profession,date,jour,mois,annee,lon,lat,ville,pays,note
p1,Bob,Data science,2020-09-03,3,9,2020,41.5,12.8,Aix,France,3
p2,Alice,Architecte,2020-09-01,1,9,2020,41.6,12.8,Aix,France,5
p13,Bob,Data science,2020-02-01,1,2,2020,32.1,12.1,Lille,France,5
p14,Carole,Data science,2020-02-01,1,2,2020,32.1,12.1,Lille,France,2
p12,Alice,Architecte,2020-02-01,1,2,2020,32.1,12.1,Lille,France,3
p7,Alice,Architecte,2020-10-05,5,10,2020,41.8,12.8,Marseille,France,4
p6,Alice,Architecte,2020-09-05,5,9,2020,41.6,12.7,Nice,France,4
p9,David,Architecte,2019-12-25,25,12,2019,30.1,10.1,Paris,France,1
p8,Carole,Data science,2019-12-25,25,12,2019,30.1,10.1,Paris,France,2
p16,Alice,Architecte,2019-12-25,25,12,2019,30.1,10.1,Paris,France,4


### L'opération GROUP BY ROLLUP

#### Rollup sur la dimension temporelle
Rassembler les agrégations à tous les niveaux de la dimension sur la date: par jour, par mois et par année et global (ie, qqsoit la date)

In [ ]:
%sql
select annee, mois, jour, count(*) as nb_visite
from Visite v, DateDimension d
where v.date = d.date
group by rollup(annee, mois, jour)
order by annee, mois, jour

annee,mois,jour,nb_visite
null,null,null,17
2019,null,null,6
2019,11,null,1
2019,11,11,1
2019,12,null,5
2019,12,25,4
2019,12,26,1
2020,null,null,11
2020,2,null,4
2020,2,1,3


#### Rollup sur la dimension géographique

On restructure la clé composée de (lon,lat) en une clé ayant un seul attribut gps.

In [ ]:
%sql
create or replace temp view VisiteGPS as
select v.photoID, v.personID,  v.date,  (v.lon, v.lat) as gps, v.note
from Visite v;

select *
from VisiteGPS;

photoID,personID,date,gps,note
p1,Bob,2020-09-03,"List(41.5, 12.8)",3
p2,Alice,2020-09-01,"List(41.6, 12.8)",5
p3,Bob,2020-09-04,"List(41.6, 13.2)",1
p4,Bob,2020-09-04,"List(40.1, 12.0)",2
p5,Alice,2020-09-04,"List(40.1, 12.0)",2
p6,Alice,2020-09-05,"List(41.6, 12.7)",4
p7,Alice,2020-10-05,"List(41.8, 12.8)",4
p8,Carole,2019-12-25,"List(30.1, 10.1)",2
p9,David,2019-12-25,"List(30.1, 10.1)",1
p10,Eva,2019-12-25,"List(30.1, 10.1)",5


In [ ]:
%sql
create or replace temp view PlaceGPS as
select (p.lon, p.lat) as gps, p.ville, p.pays
from Place p;

select *
from PlaceGPS;

gps,ville,pays
"List(41.5, 12.8)",Aix,France
"List(41.6, 12.8)",Aix,France
"List(41.6, 12.7)",Nice,France
"List(41.8, 12.8)",Marseille,France
"List(41.6, 13.2)",Rome,Italie
"List(40.1, 12.0)",Oslo,Norvege
"List(30.1, 10.1)",Paris,France
"List(31.1, 10.1)",StDenis,France
"List(32.1, 12.1)",Lille,France
"List(49.1, 10.1)",Pau,France


Le nombre de Visites par lieu (_ie._, position GPS), ville et pays

In [ ]:
%sql
select p.pays, p.ville, v.gps, count(*) as nb_visite
from VisiteGPS v, PlaceGPS p
where v.gps = p.gps
group by rollup( p.pays, p.ville, v.gps )
order by pays, ville, gps

pays,ville,gps,nb_visite
null,null,null,17
France,null,null,14
France,Aix,null,2
France,Aix,"List(41.5, 12.8)",1
France,Aix,"List(41.6, 12.8)",1
France,Lille,null,3
France,Lille,"List(32.1, 12.1)",3
France,Marseille,null,1
France,Marseille,"List(41.8, 12.8)",1
France,Nice,null,1


### L'opération GROUP BY CUBE et manipulation d'un cube
Rassembler les agrégations des Visites sur l'année, la profession, et le pays. 
On appelle le résultat : Cube1

In [ ]:
%sql
create or replace temporary view Cube1 as
select annee, pays, profession, round(avg(note), 1) as noteMoyenne, count(*) as nbVisite
from VisitesDetail
group by cube (annee, pays, profession)
order by annee, pays, profession;

cache table Cube1;

select *
from Cube1;

annee,pays,profession,noteMoyenne,nbVisite
null,null,null,3.1,17
null,null,Architecte,3.5,8
null,null,Data science,2.3,7
null,null,Vendeuse,4.0,2
null,France,null,3.4,14
null,France,Architecte,3.7,7
null,France,Data science,2.6,5
null,France,Vendeuse,4.0,2
null,Italie,null,1.0,1
null,Italie,Data science,1.0,1


Projection agrégative : sélectionner certaines dimensions du cube

projeter sur la dimension profession

In [ ]:
%sql
select profession, notemoyenne, nbVisite 
from Cube1
where profession is not null
and pays is null
and annee is null

profession,notemoyenne,nbVisite
Architecte,3.5,8
Data science,2.3,7
Vendeuse,4.0,2


projeter sur les dimensions profession et pays

In [ ]:
%sql
select profession, pays, notemoyenne, nbVisite 
from Cube1 c
where profession is not null
and pays is not null
and annee is null
order by profession, pays

profession,pays,notemoyenne,nbVisite
Architecte,France,3.7,7
Architecte,Norvege,2.0,1
Data science,France,2.6,5
Data science,Italie,1.0,1
Data science,Norvege,2.0,1
Vendeuse,France,4.0,2


Opération **Dice** : critère de sélection sur la mesure

In [ ]:
%sql
select * 
from Cube1
where noteMoyenne >=3.5 and nbVisite < 8

annee,pays,profession,noteMoyenne,nbVisite
null,null,Vendeuse,4.0,2
null,France,Architecte,3.7,7
null,France,Vendeuse,4.0,2
2019,null,Vendeuse,4.0,2
2019,France,Vendeuse,4.0,2
2020,null,Architecte,3.8,6
2020,France,Architecte,4.2,5


Opération **Slice**: critère de sélection sur des dimensions

In [ ]:
%sql
select *
from Cube1
where pays in ('Norvege', 'France')
and annee > 2018

annee,pays,profession,noteMoyenne,nbVisite
2019,France,null,2.7,6
2019,France,Architecte,2.5,2
2019,France,Data science,1.5,2
2019,France,Vendeuse,4.0,2
2020,France,null,3.9,8
2020,France,Architecte,4.2,5
2020,France,Data science,3.3,3
2020,Norvege,null,2.0,2
2020,Norvege,Architecte,2.0,1
2020,Norvege,Data science,2.0,1


### Requêtes avec fonctions de fenêtrage

#### Requete avec fenêtre de taille croissante

Numéroter les tuples

In [ ]:
%sql
select v.personId, v.photoId, v.note, 
       row_number() over (order by personId, photoId) as numeroPhoto
from Visite v

personId,photoId,note,numeroPhoto
Alice,p12,3,1
Alice,p16,4,2
Alice,p17,5,3
Alice,p2,5,4
Alice,p5,2,5
Alice,p6,4,6
Alice,p7,4,7
Bob,p1,3,8
Bob,p13,5,9
Bob,p3,1,10


Classement des visites par note décroissantes

In [ ]:
%sql
select v.personId, v.photoId, v.note, rank() over (order by note desc) as rang
from Visite v

personId,photoId,note,rang
Alice,p2,5,1
Eva,p10,5,1
Bob,p13,5,1
Alice,p17,5,1
Alice,p6,4,5
Alice,p7,4,5
Alice,p16,4,5
Bob,p1,3,8
Eva,p11,3,8
Alice,p12,3,8


###### top K
 Le top 5 des Visites avec les meilleures notes.
Il peut y avoir plus de k visites dans le résultat à cause des ex aequos.

In [ ]:
%sql 
With Classement as (
Select v.personId, v.photoId, v.note, rank() over (order by note desc) as rang
From Visite v
)
Select * from Classement 
where rang <=5
order by rang

personId,photoId,note,rang
Alice,p2,5,1
Alice,p17,5,1
Eva,p10,5,1
Bob,p13,5,1
Alice,p6,4,5
Alice,p7,4,5
Alice,p16,4,5


Classement dense

In [ ]:
%sql
select v.personId, v.photoId, v.note, dense_rank() over (order by note desc) as rang_dense
from Visite v

personId,photoId,note,rang_dense
Alice,p2,5,1
Eva,p10,5,1
Bob,p13,5,1
Alice,p17,5,1
Alice,p6,4,2
Alice,p7,4,2
Alice,p16,4,2
Bob,p1,3,3
Eva,p11,3,3
Alice,p12,3,3


#### Fenêtrage et partitionnement : Partition by

##### Partition by 
Une partition est définie par un sous-ensemble des attributs projetés dans le select.

Une fenêtre par partition.

Le classement **par ville** des visites les mieux notées

In [ ]:
%sql
SELECT v.photoId, v.personID, v.ville, 
       rank() OVER( PARTITION BY ville order by note) as rang
FROM VisitesDetail v
order by ville, rang

photoId,personID,ville,rang
p1,Bob,Aix,1
p2,Alice,Aix,2
p14,Carole,Lille,1
p12,Alice,Lille,2
p13,Bob,Lille,3
p7,Alice,Marseille,1
p6,Alice,Nice,1
p4,Bob,Oslo,1
p5,Alice,Oslo,1
p9,David,Paris,1


Le classement **par pays**  des villes ayant de plus de visites

In [ ]:
%sql 
Select pays, ville, count(*) as nbVisite, 
       rank() over (partition by pays order by count(*) desc) as rang
From VisitesDetail v
Group by pays, ville
Order by pays, rang

pays,ville,nbVisite,rang
France,Paris,4,1
France,Lille,3,2
France,Pau,2,3
France,Aix,2,3
France,Marseille,1,5
France,Nice,1,5
France,StDenis,1,5
Italie,Rome,1,1
Norvege,Oslo,2,1


# QUESTIONS

### Question 1 : temps écoulé

Pour chaque visite de chaque personne, le temps écoulé depuis la première visite de cette personne. Exprimer le temps écoulé par un intervalle de temps résultant d'une différence entre deux dates. L'intervalle indique le nombre de mois et le nombre de jours.

Le schema attendu est (photoID , personID, date, datePremiereVisiste, temps_ecoule)

Astuce: commencer par compléter chaque visite avec la date de la première visite de la personne.

In [ ]:
%sql
with T as(
select personID, min(date) as datePremiereVisite
      from VisitesDetail
      group by personID
)
select v.photoID, v.personID, v.ville, v.date, T.datePremiereVisite, 
cast(v.date-T.datePremiereVisite as string) as temps_ecoule,
DATEDIFF(v.date, T.datePremiereVisite) as jour_ecoule
from VisitesDetail v
LEFT JOIN T ON v.personID = T.personID;

photoID,personID,ville,date,datePremiereVisite,temps_ecoule,jour_ecoule
p1,Bob,Aix,2020-09-03,2020-02-01,7 months 2 days,215
p2,Alice,Aix,2020-09-01,2019-12-25,8 months 7 days,251
p3,Bob,Rome,2020-09-04,2020-02-01,7 months 3 days,216
p4,Bob,Oslo,2020-09-04,2020-02-01,7 months 3 days,216
p5,Alice,Oslo,2020-09-04,2019-12-25,8 months 10 days,254
p6,Alice,Nice,2020-09-05,2019-12-25,8 months 11 days,255
p7,Alice,Marseille,2020-10-05,2019-12-25,9 months 10 days,285
p8,Carole,Paris,2019-12-25,2019-11-11,1 months 14 days,44
p9,David,Paris,2019-12-25,2019-12-25,0 seconds,0
p10,Eva,Paris,2019-12-25,2019-12-25,0 seconds,0


### Question 2: Requête avec fenêtre glissante de taille fixe

Pour chaque visite de chaque personne, le nombre de jours depuis la visite précédente de cette personne. 

Le schéma attendu est (photoID, personID, date, precedenteDate, nbJours)

Astuce: penser à utiliser la fonction datediff pour comparer deux dates et obtenir un nombre de jours

In [ ]:
%sql
-- à compléter (solution dans TP2_solutions)
create or replace temporary view Q2 as
select v.photoID, v.personID, v.date, 
min(v.date) over (partition by PersonID order by v.date rows between 1 preceding and current row) as precedenteDate,
datediff(v.date, min(v.date) over (partition by PersonID order by v.date rows between 1 preceding and current row)) as nbJours
from VisitesDetail v;

cache table Q2;

select *
from Q2;

photoID,personID,date,precedenteDate,nbJours
p9,David,2019-12-25,2019-12-25,0
p16,Alice,2019-12-25,2019-12-25,0
p12,Alice,2020-02-01,2019-12-25,38
p17,Alice,2020-02-02,2020-02-01,1
p2,Alice,2020-09-01,2020-02-02,212
p5,Alice,2020-09-04,2020-09-01,3
p6,Alice,2020-09-05,2020-09-04,1
p7,Alice,2020-10-05,2020-09-05,30
p15,Carole,2019-11-11,2019-11-11,0
p8,Carole,2019-12-25,2019-11-11,44


## Question 3 : Semaine glissante
Pour chaque jour de visite de chaque personne, le nombre de visite sur 7 jours glissants.

Schema (personID, date, nbVisite7jours)

In [ ]:
%sql
-- à compléter (solution dans TP2_solutions)
select v.personID, v.date, 
count(v.date) over (partition by PersonID order by v.date range between 7 preceding and current row) as nbVisite7jours
from VisitesDetail v

personID,date,nbVisite7jours
Alice,2019-12-25,1
Alice,2020-02-01,1
Alice,2020-02-02,2
Alice,2020-09-01,1
Alice,2020-09-04,2
Alice,2020-09-05,3
Alice,2020-10-05,1
Bob,2020-02-01,1
Bob,2020-09-03,1
Bob,2020-09-04,3


## Question 4 : Trajets
On veut enrichir les visites avec une information de trajet.
Un trajet représente toutes les visites d'une personne telles que le nombre de jours entre deux visites consécutives soit inférieur à une semaine.

Schema : (photoID, personID, date, nbJours, numTrajet)

Astuces :
utiliser la réponse à la question 2. La fonction if dans la clause select peut indiquer si la visite est le début du trajet suivant ou non.

In [ ]:
%sql
-- à compléter (solution dans TP2_solutions)
with T as(
      select photoID, personID, if(nbJours>6, 1, 0) as isTrajet
      from Q2
      group by photoID, personID, isTrajet
)

select q.photoID, q.personID, q.date, q.nbJours, sum(T.isTrajet) over (partition by q.personID order by q.date) as numTrajet
from Q2 q
INNER JOIN T ON q.personID = T.personID and q.photoID=T.photoID;

photoID,personID,date,nbJours,numTrajet
p9,David,2019-12-25,0,0
p16,Alice,2019-12-25,0,0
p12,Alice,2020-02-01,38,1
p17,Alice,2020-02-02,1,1
p2,Alice,2020-09-01,212,2
p5,Alice,2020-09-04,3,2
p6,Alice,2020-09-05,1,2
p7,Alice,2020-10-05,30,3
p15,Carole,2019-11-11,0,0
p8,Carole,2019-12-25,44,1


### 4 b)
Ajouter la condition: un trajet ne peut pas durer plus de 3 jours au total

In [ ]:
def numerote():
  

In [ ]:
%sql
select photoID, personID, date, nbJours, sum(nbJours) over (partition by personID order by date range between 3 preceding and 0 following)
from Q2

photoID,personID,date,nbJours,sum(nbJours) OVER (PARTITION BY personID ORDER BY date ASC NULLS FIRST RANGE BETWEEN 3 PRECEDING AND 0 FOLLOWING)
p9,David,2019-12-25,0,0
p16,Alice,2019-12-25,0,0
p12,Alice,2020-02-01,38,38
p17,Alice,2020-02-02,1,39
p2,Alice,2020-09-01,212,212
p5,Alice,2020-09-04,3,215
p6,Alice,2020-09-05,1,4
p7,Alice,2020-10-05,30,30
p15,Carole,2019-11-11,0,0
p8,Carole,2019-12-25,44,44


## Question 5 : IMDB

Proposer des requetes OVER PARTITION BY  sur la base IMDB